In [ ]:
%config InlineBackend.figure_format = 'retina'

# Testing Enviornment

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np

## Running tests

The following section unit-tests most of the code written for the proof of concept prototype.

In [ ]:
%run ../test/test_algorithms.py

In [ ]:
%run ../test/test_dataloader.py

In [ ]:
%run ../test/test_predictorsI.py

In [ ]:
%run ../test/test_predictorsII.py

In [ ]:
%run ../test/test_activate.py

## Running example of the system

In [ ]:
%run ../consensus/algorithms.py

In [ ]:
%run ../tools/dataloader.py

In [ ]:
test = DataLoader('aapl', '2009-01-01', '2010-02-10')

In [ ]:
prices = test.get_close()

In [ ]:
prices

In [ ]:
#prices = np.array(prices)
#len(prices)

In [ ]:
prices

In [ ]:
%run ../tools/predictorsI.py

In [ ]:
op0 = BasicUnivariatePredictor(prices, 24, 30)
op1 = BasicUnivariatePredictor(prices, 25, 7)
op2 = BasicUnivariatePredictor(prices, 25, 7)
op3 = BasicUnivariatePredictor(prices, 25, 7)

In [ ]:
op0.create_bilstm()

In [ ]:
op0.model_blueprint()

In [ ]:
op0.fit_model(150)

In [ ]:
op0.show_performance()

In [ ]:
op0.save_model()

In [ ]:
oyea = prices[-26:-1]
#oyea = X[-1]
#oyea

In [ ]:
nice = op0.predict(oyea)
nice

In [ ]:
op1.create_lstm()

In [ ]:
op1.model_blueprint()

In [ ]:
op1.fit_model(10)

In [ ]:
op1.show_performance()

In [ ]:
nice = op1.predict(oyea)
nice

In [ ]:
op2.create_cnn()

In [ ]:
op2.model_blueprint()

In [ ]:
op2.fit_model(10)

In [ ]:
op2.show_performance()

In [ ]:
nice = op2.predict(oyea)
nice

In [ ]:
op3.create_mlp()

In [ ]:
op3.model_blueprint()

In [ ]:
op3.fit_model(100)

In [ ]:
op3.show_performance()

In [ ]:
oyea = prices[-26:-1]
#oyea = X[-1]
#oyea

In [ ]:
nice = op3.predict(oyea)
nice

In [ ]:
%run ../tools/predictorsII.py

In [ ]:
oo = UnivariatePredictorII(prices, 7)

In [ ]:
oo.fit_neural_model(100,"D")

In [ ]:
oo.show_performance_neural()

In [ ]:
oo.predict_neural()

In [ ]:
oo.fit_prophet_model()

In [ ]:
oo.show_performance_prophet()

In [ ]:
oo.predict_prophet()

In [ ]:
%run ../tools/predictorsIII.py

In [ ]:
len(prices)

In [ ]:
op4 = HybridUnivariatePredictor(prices,2, 24, 7)

In [ ]:
op4.create_cnnlstm()

In [ ]:
op4.model_blueprint()

In [ ]:
op4.fit_model(10)

In [ ]:
op4.show_performance()

In [ ]:
oyea = prices[-25:-1]
#oyea = X[-1]
#oyea

In [ ]:
nice = op4.predict(oyea)
nice
#noice = pd.DataFrame(nice, columns=['yea'])
#noice = nice.reshape(20, 1)
#noice = pd.DataFrame(noice, columns=['yea'])

In [ ]:
nice.plot()

## Whole system test - I am alive v.2

In [9]:
%run ../tools/dataloader.py
%run ../system/activate.py

In [ ]:
training = DataLoader('aapl', '2009-01-01', '2010-05-01')

In [ ]:
training = training.get_close()

In [ ]:
#real = DataLoader('aapl', '2010-05-01', '2010-05-28')

In [ ]:
#real = real.get_close()

In [ ]:
#len(real)

In [3]:
predict = DataLoader('aapl', '2010-06-01', '2010-09-01')

In [4]:
predict = predict.get_close()

In [5]:
predict_req, real = data_prep(predict, 24, 30)

In [ ]:
real

In [ ]:
final_df1 = individual_predictors1(training, predict_req, 30)

In [ ]:
final_df2 = individual_predictors2(training, predict_req, 30)

In [ ]:
final_df3 = individual_predictors3(training, predict_req, 30)

In [ ]:
final_df4 = individual_predictors4(training, predict_req, 30)

In [ ]:
final_df5 = individual_predictors5(training, 20)

In [10]:
final_df6 = individual_predictors_pretrained1(predict_req, 30)

In [11]:
final_df6

,CNN-LSTM,MLP
0,9.116774,9.221597
1,8.952804,9.571225
2,8.725756,8.856209
3,8.563080,10.683270
4,8.261903,9.079777
5,7.963663,9.712100
6,7.678870,9.424383
7,6.938310,8.293983
8,6.712652,9.337017
9,6.351397,9.581328


### System Disagreement

In [ ]:
system_disagreement(final_df1)

In [ ]:
system_disagreement(final_df2)

In [ ]:
system_disagreement(final_df3)

In [ ]:
system_disagreement(final_df4)

In [ ]:
system_disagreement(final_df5)

### System consensus

In [ ]:
algos1 = consensus(final_df1, real)

In [ ]:
algos2 = consensus(final_df2, real)

In [ ]:
algos3 = consensus(final_df3, real)

In [ ]:
algos4 = consensus(final_df4, real)

In [ ]:
algos5 = consensus(final_df5, real)

In [ ]:
ui1 = combined_frame(final_df1, algos1, real)

In [ ]:
ui2 = combined_frame(final_df2, algos2, real)

In [ ]:
ui3 = combined_frame(final_df3, algos3, real)

In [ ]:
ui4 = combined_frame(final_df4, algos4, real)

In [ ]:
ui5 = combined_frame(final_df5, algos5, real)

In [ ]:
yu = all_stats_frame(ui5, final_df5)

In [ ]:
yu

In [ ]:
mse_score(ui5)

In [ ]:
mse_log_score(ui4)

In [ ]:
mae_score(ui1)

In [ ]:
mse_score(ui2)

In [ ]:
mse_log_score(ui1)

In [ ]:
mae_score(ui2)

In [ ]:
mse_score(ui3)

In [ ]:
mse_log_score(ui3)

In [ ]:
mae_score(ui3)

In [ ]:
plot_performance(ui1)

In [ ]:
plot_performance(ui2)

In [ ]:
plot_performance(ui3)

In [ ]:
plot_performance(ui4)

In [ ]:
plot_performance(ui5)

In [ ]:
ui5

In [ ]:
training = DataLoader('aapl', '2000-01-01', '2010-05-01')

In [ ]:
training = training.get_close()

In [ ]:
motest = HybridUnivariatePredictor(training,2, 24, 30)

In [ ]:
motest.create_cnnlstm()

In [ ]:
motest.fit_model(300)

In [ ]:
motest.show_performance()

In [ ]:
motest.save_model()

In [ ]:
moload = HybridUnivariatePredictor(training,2, 24, 30)

In [ ]:
moload.load_model('../pretrained/model')

In [ ]:
moload.model_blueprint()

In [ ]:
moload.predict(training[-25:-1])

In [ ]:
test = DataFrame()

In [ ]:
np.array(None) != None

In [ ]:
type(test)

In [ ]:
uuu = np.empty(shape=[3, 10])

In [ ]:
uuu

In [ ]:
uuu.shape[1]